In [53]:
#Task: text summarization of entertainment BBC news articles
import os
import pandas as pd
from datasets import load_dataset, Dataset
import string
import evaluate
import torch

In [54]:
#load and preprocess data
#dataset used: https://www.kaggle.com/datasets/pariza/bbc-news-summary
dataset = load_dataset("text", data_files={"train" : "data/BBC News Summary/News Articles/entertainment/*.txt", "test" : "data/BBC News Summary/Summaries/entertainment/*.txt"})

def clean_text(text):
    text = " ".join(text.split())

    text = text.lower()

    return text

def preprocess(data):
    cleaned_data = [clean_text(text) for text in data["text"]]

    return {"text" : cleaned_data}

processed_data = dataset.map(preprocess, batched=True)

train_df = pd.DataFrame(processed_data["train"])
train_df.drop_duplicates(subset=["text"], keep="first", inplace=True)

processed_data["train"] = train_df

processed_data["train"] = Dataset.from_pandas(train_df)




In [55]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
tokenizer = AutoTokenizer.from_pretrained("t5-small") 
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small") 
def preprocess_function(examples): 
    model_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=150)
    model_inputs["labels"] = labels["input_ids"]
    #return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512) 
    return model_inputs
tokenized_dataset = processed_data.map(preprocess_function, batched=True)

Map: 100%|██████████| 2005/2005 [00:02<00:00, 998.73 examples/s] 


In [ ]:
#train model
from transformers import Trainer, TrainingArguments 
training_args = TrainingArguments( output_dir="./results", evaluation_strategy="epoch", learning_rate=2e-5, num_train_epochs=3, weight_decay=0.01,) 
trainer = Trainer( model=model, args=training_args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["test"], ) 
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.087900,0.086033
2,0.039000,0.068008
3,0.035700,0.063675


TrainOutput(global_step=6015, training_loss=0.2505281498901862, metrics={'train_runtime': 932.6565, 'train_samples_per_second': 6.449, 'train_steps_per_second': 6.449, 'total_flos': 814080935854080.0, 'train_loss': 0.2505281498901862, 'epoch': 3.0})

In [ ]:
#ROUGE score

#torch.cuda.empty_cache()
#torch.cuda.reset_max_memory_allocated()

rouge = evaluate.load("rouge")

def rouge_score(preds, refs):
    predictions = [tokenizer.decode(p, skip_special_tokens=True) for p in preds]
    references = [tokenizer.decode(r, skip_special_tokens=True) for r in refs]
    result = rouge.compute(predictions=predictions, references=references)
    return result

predictions = trainer.predict(tokenized_dataset["test"])
r_score = rouge_score(predictions.predictions, predictions.label_ids)

print(f"ROUGE Score: {r_score}")

c:\Python311\Lib\site-packages\torch\cuda\memory.py:391: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.44 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 3.27 GiB is allocated by PyTorch, and 3.17 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)